In [1]:
%matplotlib inline

In [2]:
import subprocess
from matplotlib import pyplot as plt

import json

In [3]:
def call_RNAfold(sequence):
    MyOut = subprocess.Popen(['RNAfold', '-p', '--noPS', '--constraint'],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
    stdout, stderr = MyOut.communicate(input=str.encode(sequence))
    return stdout

def get_energy_RNAfold(stdout_string):
    temp = stdout_string.decode("utf-8") 
    energy_line = temp.split('\n')[-5]
    energy_val = energy_line[energy_line.index(' '):]
    energy_val = energy_val.strip().strip('()').strip()
    
    mfe_line = temp.split('\n')[-4]
    mfe_val = mfe_line[energy_line.index(' '):]
    mfe_val = mfe_val.strip().strip('[]').strip()
    return float(energy_val), float(mfe_val)    

# Establish some basic dataframes that should contain all the information we need

In [4]:
import re
import pandas as pd
from Bio import SeqIO

genome_location = './ecoli.gb'
genome_type = 'genbank'
###Read genome
genome = list(SeqIO.parse(genome_location, genome_type))

###Good to double check. If this fails on a future genome we'll have to decide what to do
assert len(genome) == 1
genome = genome[0]

###Set up empty lists for now
###These are for the gene dataframe
loci = []
starts = []
stops = []
strands = []
###These are for the start codon dataframe
loci_long = []
ATG_indices = []


###And iterate through the genome
for feature in genome.features:
    if feature.type == 'CDS':
        ###Filter out the weirdos
        if 'pseudo' in feature.qualifiers or 'ribosomal_slippage' in feature.qualifiers:
            continue
        ###Make sure that no locus_tag appears twice in the genome
        ###If this happens on some future genome then we'll have to figure out what to do about it
        locus_tag = feature.qualifiers['locus_tag'][0]
        assert locus_tag not in loci
        ###Extract the coding sequence
        seq = str(feature.extract(genome.seq))
        ###Double check that all coding sequences are a multiple of 3
        assert len(seq)%3 == 0
        assert feature.location.end - feature.location.start == len(seq)
        ###Append some basic data
        loci.append(locus_tag)
        starts.append(feature.location.start)
        stops.append(feature.location.end)
        strands.append(feature.location.strand)
        
        ###Now on to ATGS
        ###This is a quick/efficient way to find the occurrence of all substrings within a string
        ATG_locs = [m.start() for m in re.finditer('ATG', seq)]
        ###Iterate through all ATG indices and append the data
        for loc in ATG_locs:
            loci_long.append(locus_tag)
            ATG_indices.append(loc)

###Create and clean up a gene level dataframe
col_names = ['start', 'stop', 'strand']
gene_df = pd.DataFrame(list(zip(starts, stops, strands)), columns=col_names, index=loci)

###Create and clean up a start codon data frame
###Now make this all into a nice and easy pandas dataframe
col_names = ['locus_tag', 'ATG_index']
start_df = pd.DataFrame(list(zip(loci_long, ATG_indices)), columns=col_names)

###These last two columns we might want are kind of unnecessary/re-dundant but here is a quick way
###to calculate them
###First set the default as Internal
start_df['location'] = 'Internal'
###And make it External for every ATG_index of zero
start_df.at[start_df[start_df['ATG_index'] == 0].index, 'location'] = 'External'

###Same thing here. Set the default as False
start_df['within_frame'] = False
###And now say if the index divided by 3 has no remainder, set it to True
start_df.at[start_df[start_df['ATG_index']%3 == 0].index, 'within_frame'] = True
###And if we want the External ATGs to not have a within_frame value we can now just say
start_df.at[start_df[start_df['location'] == 'External'].index, 'within_frame'] = ''


In [12]:
###Writing pandas dataframes are crazy easy and so much easier/more common than csvwrite/reader
###Within the sep argument, you can make it comma separated, tab separated, etc.
###I usually prefer tab separated files (tsv) for no particular reason.
###The index=False statement can be True or False. It really only affects the way you read the files
###later on

gene_df.to_csv('../Data/ecoli_gene_df.tsv', sep='\t', index=False)

# Let's look at those dataframes a bit

In [5]:
start_df.head()

,locus_tag,ATG_index,location,within_frame
0,b0001,0,External,
1,b0002,0,External,
2,b0002,37,Internal,False
3,b0002,79,Internal,False
4,b0002,147,Internal,True


In [6]:
print(start_df.shape)

(77344, 4)


In [7]:
gene_df.head()

,start,stop,strand
b0001,189,255,1
b0002,336,2799,1
b0003,2800,3733,1
b0004,3733,5020,1
b0005,5233,5530,1


In [22]:
print(gene_df.shape)

(4239, 3)


# Now calculate all of the relevant folding energies and add them to the dataframe

As (currently) written, this code should extract 50 nt's before the start codon, the start codon itself, and then 47 nt's downstream of the start codon (for a total of 100 nt's). This decision is entirely arbitrary and the source of much contention/error but no one knows a better way to do it.

From that 100 nt fragment, we can calculate the folding energy (both the minimum folding energy or the folding energy of the thermodynamic ensemble). We can also add in a constraint that some region has to be unpaired (like the SD region) and ask how the folding energy changes between the unconstrained and constrained values. 

Note: On my laptop, for the E. coli genome with ~80,000 ATGs, this code took about 2 hours to run. So not bad all things considered but will get annoying as we consider GTG, TTG, etc. codons one day.

In [18]:
flanking_bases = 50
seq_len = 100

###Important to check this later. These are the basepair constraints used for calculating
###free energy under the condition that we constrain the SD region to be open
constraint = ('.'*34) + ('x'*12)+ ('.'*54)
assert len(constraint) == seq_len

###Iterate through all the start codons
for index in start_df.index[:]:
    ###Get this info directly from the index
    locus = start_df.at[index, 'locus_tag']
    start_index = start_df.at[index, 'ATG_index']
    ###And this info from the gene_df
    strand = gene_df.at[locus, 'strand']
    ###Get the sequence (strand dependent)
    if strand == 1:
        start = gene_df.at[locus, 'start']
        ###Ensure that I have found an ATG
        if genome.seq[start+start_index:start+start_index+3] != 'ATG':
            print('Something is rotten in the state of denmark')
            continue
        seq = genome.seq[start+start_index-flanking_bases:start+start_index+flanking_bases]

    elif strand == -1:
        start = gene_df.at[locus, 'stop']
        if genome.seq[start-start_index-3:start-start_index].reverse_complement() != 'ATG':
            print('Something is rotten in the state of denmark, on the other side')
            continue    
        seq = genome.seq[start-start_index-flanking_bases:start-start_index+flanking_bases].reverse_complement()
    
    ###Just like quadruple checking that this is true
    assert len(seq) == seq_len
    
    ###Call RNAfold on the sequence!
    RNA_out = call_RNAfold(str(seq))
    a, b = get_energy_RNAfold(RNA_out)
    start_df.at[index, 'Unconstrained_mfe'] = b
    start_df.at[index, 'Unconstrained_ensemble'] = a

    ###Now call it in the constrained form such that the SD region is unpaired
    RNA_out = call_RNAfold(str(seq)+'\n'+constraint)
    a, b = get_energy_RNAfold(RNA_out)
    start_df.at[index, 'Constrained_mfe'] = b
    start_df.at[index, 'Constrained_ensemble'] = a

In [19]:
start_df.head()

,locus_tag,ATG_index,location,within_frame,Unconstrained_mfe,Unconstrained_ensemble,Constrained_mfe,Constrained_ensemble
0,b0001,0,External,,-4.30,-2.0,-3.47,-1.4
1,b0002,0,External,,-20.68,-18.1,-17.03,-14.5
2,b0002,37,Internal,False,-29.91,-27.0,-24.78,-22.7
3,b0002,79,Internal,False,-43.35,-40.8,-41.45,-39.8
4,b0002,147,Internal,True,-22.77,-20.5,-17.23,-15.8


In [20]:
start_df.tail()

,locus_tag,ATG_index,location,within_frame,Unconstrained_mfe,Unconstrained_ensemble,Constrained_mfe,Constrained_ensemble
77339,b4403,496,Internal,False,-19.76,-17.8,-16.16,-15.0
77340,b4403,543,Internal,True,-22.52,-19.9,-18.91,-16.9
77341,b4403,574,Internal,False,-23.84,-21.0,-16.86,-15.1
77342,b4403,639,Internal,True,-22.39,-20.9,-17.05,-15.8
77343,b4403,658,Internal,False,-23.35,-20.8,-15.88,-14.1


In [21]:
###Write this data to a file
start_df.to_csv('../Data/ecoli_folding_energies.tsv', sep='\t', index=False)